# Introduction to computational graph and implementation of backpropagation and Gradient descent

**INSTRUCTIONS**

Each week you will be given an assignment related to the associated module. You have roughly one week to complete and submit each of them. There are 3 weekly group sessions available to help you complete the assignments. Attendance is not mandatory but recommended. However, **assignments are graded each week and not submitting them or submitting them after the deadline will give you no points**

- **FORMAT**: Jupyter notebook
- **DEADLINE**:  Sunday 21st February, 23:59


## Introduction

Last week we saw how adding regularization to SGD could reduce overfitting. This week we will try to understand what happens exactly inside your model during the training process when applying the gradient descent method, with or without regularization and momentum. To do so we will simply implement first the backprogation algorithm which is how we compute the gradients and then the gradient descent, which is the trainable parameter update in response to this gradient.


## Contents

1. Utils  

2. MiniNet  

3. Introduction to PyTorch's computational graph  
  3.1 Weight values and update  
  3.2 Gentle (but still painful) introduction to computational graph  
4. From an optimization problem to the backprogation algorithm  

5. Implementing Gradient descent in Pytorch inside the training loop   
  5.0 LetNet5  
  5.1 Manual weight optimization  
  5.2 Compare results to optim.SGD: vanilla version  
  5.3 Compare results to optim.SGD: (with Regularization)  
  5.4 Compare results to optim.SGD: (with Momentum)  
  5.5 Compare results to optim.SGD: (with Regularization and Momentum)  
6. Implementing an Optimizer in PyTorch
  
## Andrew's Videos related to today's assignent (entirely optional! just here to help you if you're confused about something specific!)

- [Computation Graph (C1W2L07)](https://www.youtube.com/watch?v=hCP1vGoCdYU&list=PLkDaE6sCZn6Ec-XTbcX1uRg2_u4xOEky0&index=13)
- [Derivatives With Computation Graphs (C1W2L08)](https://www.youtube.com/watch?v=nJyUyKN-XBQ&list=PLkDaE6sCZn6Ec-XTbcX1uRg2_u4xOEky0&index=14)
- [Gradient Descent (C1W2L04)](https://www.youtube.com/watch?v=uJryes5Vk1o&list=PLkDaE6sCZn6Ec-XTbcX1uRg2_u4xOEky0&index=10)
- [Gradient Descent For Neural Networks (C1W3L09)](https://www.youtube.com/watch?v=7bLEWDZng_M&list=PLkDaE6sCZn6Ec-XTbcX1uRg2_u4xOEky0&index=33) (includes backprogation)
- [Backpropagation Intuition (C1W3L10)](https://www.youtube.com/watch?v=yXcQ4B-YSjQ&list=PLkDaE6sCZn6Ec-XTbcX1uRg2_u4xOEky0&index=34)
- [Deep L-Layer Neural Network (C1W4L01)](https://www.youtube.com/watch?v=2gw5tE2ziqA&list=PLkDaE6sCZn6Ec-XTbcX1uRg2_u4xOEky0&index=36) (clarify notations)
- [Forward and Backward Propagation (C1W4L06)](https://www.youtube.com/watch?v=qzPQ8cEsVK8&list=PLkDaE6sCZn6Ec-XTbcX1uRg2_u4xOEky0&index=41)
- [Regularization (C2W1L04)](https://www.youtube.com/watch?v=6g0t3Phly2M&list=PLkDaE6sCZn6Hn0vK8co82zjQtt3T2Nkqc&index=4)
- [Why Regularization Reduces Overfitting (C2W1L05)](https://www.youtube.com/watch?v=NyG-7nRpsW8&list=PLkDaE6sCZn6Hn0vK8co82zjQtt3T2Nkqc&index=5)
- [Gradient Descent With Momentum (C2W2L06)](https://www.youtube.com/watch?v=k8fTYJPd3_I&list=PLkDaE6sCZn6Hn0vK8co82zjQtt3T2Nkqc&index=20)

In [2]:
import torch
from torch.optim import Optimizer
from torch import nn, optim
import torch.nn.functional as F
from torchvision import datasets, transforms
import datetime
import copy
import collections

## 1. Utils

Nothing to see in the cell below, just the definition of 3 functions we'll need later, you don't even need to read them, just know that there is:

- ``load_cifar``: load CIFAR-2 (keeping only birds and plane)
- ``training_loop``
- ``validate``

In [3]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f"Device {device}.")


def load_cifar(data_path='../data/'):
    """
    Return CIFAR-2 keeping only birds and plane
    """
    cifar10_train = datasets.CIFAR10(
        data_path,       
        train=True,      
        download=True,   
        transform=transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.4915, 0.4823, 0.4468),
                                (0.2470, 0.2435, 0.2616))
        ]))

    cifar10_val = datasets.CIFAR10(
        data_path, 
        train=False,
        download=True,
        transform=transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.4915, 0.4823, 0.4468),
                                (0.2470, 0.2435, 0.2616))
        ]))

    label_map = {0: 0, 2: 1}
    class_names = ['airplane', 'bird']

    cifar2_train = [(img, label_map[label]) for img, label in cifar10_train if label in [0, 2]]
    cifar2_val = [(img, label_map[label]) for img, label in cifar10_val if label in [0, 2]]
    print('Size of the training dataset: ', len(cifar2_train))
    print('Size of the validation dataset: ', len(cifar2_val))

    return cifar2_train, cifar2_val

def training_loop(n_epochs, optimizer, model, loss_fn, train_loader):
    """
    Train our model
    """
    model.train()
    for epoch in range(1, n_epochs + 1):
        loss_train = 0.0
        for imgs, labels in train_loader:
            imgs = imgs.to(device=device) 
            labels = labels.to(device=device)

            outputs = model(imgs)
            loss = loss_fn(outputs, labels)

            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            loss_train += loss.item()

        if epoch == 1 or epoch % 10 == 0:
            print('{}  |  Epoch {}  |  Training loss {:.3f}'.format(
                datetime.datetime.now(), epoch,
                loss_train / len(train_loader)))

def validate(model, train_loader, val_loader):
    """
    Plot training and validation accuracy
    """
    model.eval()
    accdict = {}
    for name, loader in [("train", train_loader), ("val", val_loader)]:
        correct = 0
        total = 0

        with torch.no_grad():
            for imgs, labels in loader:
                imgs = imgs.to(device=device)
                labels = labels.to(device=device)

                outputs = model(imgs)
                _, predicted = torch.max(outputs, dim=1)
                total += labels.shape[0]
                correct += int((predicted == labels).sum())

        print("Accuracy {}: {:.2f}".format(name , correct / total))
        accdict[name] = correct / total
    return accdict

cifar2_train, cifar2_val = load_cifar()

Device cpu.
Files already downloaded and verified
Files already downloaded and verified
Size of the training dataset:  10000
Size of the validation dataset:  2000


## 2. MiniNet

Here is the definition of a very simple MLP that we will use in the first part of this assignment. It corresponds to the following network (I tried to follow Andrew's notation):

![MiniNet architecture](MiniNet.jpeg)

As you will use this implementation, it might be useful to read the code bellow

In [4]:
class MiniNet(nn.Module):
    def __init__(self):
        super().__init__() 
        self.fc1 = nn.Linear(in_features=2, out_features=3)
        self.fc2 = nn.Linear(in_features=3, out_features=2)

        # Where we will store our neuron values
        # - z: before activation function 
        # - a: after activation function (a=f(z))
        self.z = {str(i) : None for i in range(1,3)}
        self.a = {str(i) : None for i in range(3)}

        # Where we will store the gradients for our custom backpropagation algo
        self.dL_dw = {str(i) : None for i in range(1,3)}
        self.dL_db = {str(i) : None for i in range(1,3)}

        # Derivatives of our activation functions
        self.df = {str(i) : None for i in range(1,3)}
        self.df['1'] = lambda x : torch.div(1, torch.cosh(x)**2)
        self.df['2'] = lambda x : torch.div(1, torch.cosh(x)**2)
        
    def forward(self, x):
        # The first dimension of the input must be the batch size
        out = x.view(-1, 2)

        # Input layer
        self.a['0'] = out
        
        # First layer (hidden layer)
        self.z['1'] = self.fc1(out)
        self.a['1'] = torch.tanh(self.z['1'])
        
        # Second layer (output layer)
        self.z['2'] = self.fc2(self.a['1'])
        self.a['2'] = torch.tanh(self.z['2'])

        return self.a['2']

## 3. Introduction to PyTorch's computational graph

### 3.1 Weight values and update

Get more familiar with models in Pytorch. Look at the cell below to understand how to:

- Access neuron values
- Access parameter values
- Update parameters

In [5]:
model = MiniNet()

def print_parameters(model):
    """
    Print trainable parameters of our MiniNet
    """
    print(" ============== Parameters ============== ")
    for name, p in model.named_parameters():
        print("\nName : ", name, "\nValue: ", p.data)

def print_neuron_values(model):
    """
    Print neuron values (a and z) of our MiniNet 
    """
    print("\n ============== Neuron values ============== ")
    print("\n -------------- Input ---------------- ")
    print("a0:            ", model.a['0'] )
    print("\n -------------- First Layer ---------------- ")
    print("z1:            ", model.z['1'] )
    print("a1 = tanh(z1): ", model.a['1'] )
    print("\n --------------  2nd Layer  ---------------- ")
    print("z2:            ", model.z['2'] )
    print("a2 = tanh(z2): ", model.a['2'] )


print_parameters(model) # We can see that all parameters are randomly initialized

# We can access the different parameters using their names as specified in model.state_dict()
print("\n ========= Print state_dict() ========= ")
print(model.state_dict())
# We can now access one of our parameters using its name
# And we can update its value using '.data' after its name
model.fc1.weight.data = torch.ones(3,2)
model.fc1.weight.data[0,0] = 42

model.fc2.bias.data[:] = torch.ones(2)
model.fc2.bias.data[0] = 42

print("\n ========== Updated parameters ==========" )

print(model.fc1.weight)
print(model.fc2.bias)


# We have not given any input to our model yet, so all neuron values should be None
print_neuron_values(model)
# Now we give some input...
input = torch.Tensor([1, 1])
model(input)
# ... and everything has been computed in the forward pass 
print_neuron_values(model)



 ============== Parameters ============== 

Name :  fc1.weight 
Value:  tensor([[0.2611, 0.5404],
        [0.0845, 0.2224],
        [0.5800, 0.4997]])

Name :  fc1.bias 
Value:  tensor([0.0394, 0.5060, 0.1236])

Name :  fc2.weight 
Value:  tensor([[-0.5603,  0.4459,  0.0277],
        [ 0.2112,  0.0214, -0.0250]])

Name :  fc2.bias 
Value:  tensor([-0.1829,  0.5082])

 ========= Print state_dict() ========= 
OrderedDict([('fc1.weight', tensor([[0.2611, 0.5404],
        [0.0845, 0.2224],
        [0.5800, 0.4997]])), ('fc1.bias', tensor([0.0394, 0.5060, 0.1236])), ('fc2.weight', tensor([[-0.5603,  0.4459,  0.0277],
        [ 0.2112,  0.0214, -0.0250]])), ('fc2.bias', tensor([-0.1829,  0.5082]))])

 ========== Updated parameters ==========
Parameter containing:
tensor([[42.,  1.],
        [ 1.,  1.],
        [ 1.,  1.]], requires_grad=True)
Parameter containing:
tensor([42.,  1.], requires_grad=True)

 ============== Neuron values ============== 

 -------------- Input ---------------- 
a0

### 3.2 Gentle (but still painful) introduction to computational graph

If you take a closer look at the output of "Updated parameters" you can see that it mentions "``requires_grad=True``". Then, if you take a closer look at the output of "neuron values" you see that it mentions "``grad_fn=<TanhBackward>``"  "``grad_fn=<AddmmBackward>``"

What is this all about? Well, it has to do with the *computational graph* which is how Pytorch manages all the operations made during the forward pass (``outputs = model(inputs)`` i.e ``forward`` method) so that it can compute all the gradients in the backward pass (``loss.backward()``) and finally update parameters accordingly when calling ``optimizer.step()``. 

Now to illustrate how necessary it is to have some understanding of this computational graph, let's try to initialize our weights in a custom way and check how easily we can mess up things.

This is okay:

- ``model.layer.param.data = new_values``
- ``model.layer.param.data[:] = new_values``

This is **NOT** okay:

- ``model.layer.param = new_values``     Raises an error (unless ``new_values`` are of nn.Parameter and not ``torch.Tensor``): ``TypeError: cannot assign 'torch.FloatTensor' as parameter 'weight' (torch.nn.Parameter or None expected)``
- ``model.layer.param[:] = new_values``  Will remove the parameter from the list of leaves and put ``CopySlices`` as gradient function

Basically, we want each of our trainable parameters (weights) to require grad ([requires_grad](https://pytorch.org/docs/stable/autograd.html?highlight=requires#torch.Tensor.requires_grad)) and to be a leaf ([is_leaf](https://pytorch.org/docs/stable/autograd.html#torch.Tensor.is_leaf)). Variables that have nothing to do with the computational graph (not a part of the network) should be detached of the computational graph. (see [detach](https://pytorch.org/docs/stable/autograd.html#torch.Tensor.detach) or [detach_](https://pytorch.org/docs/stable/autograd.html#torch.Tensor.detach_) for in place version). 

The concept of leaf might be counter intuitive in PyTorch. The fact that your weight is in the middle of your network does not mean that it should not be a leaf. It should always be a leaf. Basically weights are leaves because in the forward pass their values do not depend on the values of the input nor the weights, etc. Their values only change when you call optimizer.step() or when you initialize them manually. 

In [6]:
def weight_initialization(model):
    """
    Initialize our MiniNet's weights
    """
    model.fc1.weight.data = torch.arange(1,7, dtype=torch.float).view(3,2)/10
    model.fc1.bias.data = torch.arange(7,10, dtype=torch.float)/10
    model.fc2.weight.data = torch.arange(10,16, dtype=torch.float).view(2,3)/10
    model.fc2.bias.data = torch.arange(16,18, dtype=torch.float)/10
    return model

def check_computational_graph(model):
    """
    Make sure all trainable parameters require grad and are leaves
    """
    res = True
    # Go through layer 1,2
    for i_layer in range(1,3):
        # Each layer has a weight and bias parameter
        for param_name in ['weight', 'bias']:
            
            layer_name = "fc"+str(i_layer)
            # 'getattr(object, string variable)' is like `object.myattribut` when variable = "myattribute"
            if not getattr(getattr(model, layer_name), param_name).requires_grad:
                print(" ==== WARNING ==== ", layer_name + "." + param_name + "does not require grad!")
                print(getattr(getattr(model, layer_name), param_name))
                res = False
            if not getattr(getattr(model, layer_name), param_name).is_leaf:
                print(" ==== WARNING ==== ", layer_name + "." + param_name + " is not a leaf!")
                print(getattr(getattr(model, layer_name), param_name))
                res = False
    if res:
        print("\nAll parameters are correctly attached to the computational graph! :) ")


model = MiniNet()
check_computational_graph(model)      # So far so good, since we have not done anything yet
model = weight_initialization(model)
print_parameters(model)               # To check that parameters have been updated
check_computational_graph(model)      # To check that there are still correctly attached to the graph
model.fc1.weight[:,:] = torch.arange(1,7, dtype=torch.float).view(3,2)/10
check_computational_graph(model)      # Now fc1.weight is not a leaf anymore! (and see "grad_fn=<CopySlices>"")

# This would raise an error if uncommented
#model.fc1.weight=torch.arange(1,7, dtype=torch.float).view(3,2)/10 



All parameters are correctly attached to the computational graph! :) 
 ============== Parameters ============== 

Name :  fc1.weight 
Value:  tensor([[0.1000, 0.2000],
        [0.3000, 0.4000],
        [0.5000, 0.6000]])

Name :  fc1.bias 
Value:  tensor([0.7000, 0.8000, 0.9000])

Name :  fc2.weight 
Value:  tensor([[1.0000, 1.1000, 1.2000],
        [1.3000, 1.4000, 1.5000]])

Name :  fc2.bias 
Value:  tensor([1.6000, 1.7000])

All parameters are correctly attached to the computational graph! :) 
 ==== WARNING ====  fc1.weight is not a leaf!
Parameter containing:
tensor([[0.1000, 0.2000],
        [0.3000, 0.4000],
        [0.5000, 0.6000]], grad_fn=<CopySlices>)


## 4. From an optimization problem to the backprogation algorithm

** Optimization problem**

In optimization, (so not only specific to machine learning) the problem often boils down to minimize (or maximize) some loss function (or score). The loss function can for instance be the mean squared error:

$$L(\theta) = \frac{1}{m}\sum_{s=1}^{m}\frac{1}{2}||\mathbf{y_s} - \mathbf{\hat{y}_s}||^2_2$$ 

with:

- $m$ total number of sample in your dataset (or batch)
- $\theta$ all the parameters to be optimized ($\in \mathrm{R}^q$)
- $\mathbf{y}$ your expected result ($\in \mathrm{R}^{n \times m}$)
- $\mathbf{\hat{y}}$ your predicted result ($\in \mathrm{R}^{n \times m}$)
- $L : \mathrm{R}^q \rightarrow \mathrm{R}$ your loss function

**Gradient descent**

One way to solve this problem (still not only specific to machine learning) is to minimize the loss function iteratively using the gradient descent method. That is to say, we iteratively update all the parameters as follows (in simplest version of the gradient descent algorithm):

$$\theta = \theta - \alpha \nabla L(\theta)  $$

with 
- $\alpha$ often called "step" in optimization and "learning rate" in machine learning
- $\nabla L(\theta)$ the gradient of the loss function 

So if $\theta = \begin{bmatrix} w_{1} \cdots w_{q} \end{bmatrix}^T$  then $\nabla L(\theta) = \begin{bmatrix} \frac{\partial L}{\partial w_{1}}(\theta) \cdots \frac{\partial L}{\partial w_{q}}(\theta) \end{bmatrix}^T$

**Backpropagation algorithm**

So the only thing we need is to compute $\frac{\partial L}{\partial w_{i}}, \forall i \in [1..q]$. In machine learning, we use the backpropagation algorithm to compute $\frac{\partial L}{\partial w_{i}}$ for all the weights $w_{i}$ of a neural network. This is a recursive algorithm, from the output layer to the input layer (hence the name *backpropagation*). Again, I tried to follow Andrew's notation.

$$\frac{\partial L}{\partial w^{[l]}_{i,j}} = \delta^{[l]}_i \times a_j^{[l-1]} \qquad  \qquad \forall l \in [1 .. L] \quad \text{with }  \delta^{[l]}_i \text{ called "local gradient" }\qquad \delta^{[l]}_i = \frac{\partial L}{\partial z^{[l]}_{i}} = \frac{\partial L}{\partial a^{[l]}_{i}} \times \frac{\partial a^{[l]}_{i}}{\partial z^{[l]}_{i}}$$

For the output layer, since $a^{[L]}_{i} = \hat{y}_{i}$:

$$\delta^{[L]}_i = \frac{\partial L}{\partial z^{[L]}_{i}} = \frac{\partial L}{\partial \hat{y}_{i}} \times \frac{\partial \hat{y}_i}{\partial z^{[L]}_{i}} = e_i'(\hat{y}_i) \times f'^{[L]}_i (z^{[L]}_{i})$$

For the hidden layers (general case):

$$\delta^{[l]}_i = \frac{\partial L}{\partial z^{[l]}_{i}} = \frac{\partial L}{\partial a^{[l]}_{i}} \times \frac{\partial a^{[l]}_{i}}{\partial z^{[l]}_{i}} = \Big( \sum_{k=1}^{n^{[l+1]}} \frac{\partial a^{[l]}_{i}}{\partial a^{[l+1]}_{k}} \frac{\partial a^{[l+1]}_{k}}{\partial a^{[l]}_{i}} \Big)\times \frac{\partial a^{[l]}_{i}}{\partial z^{[l]}_{i}} = \Big( \sum_{k=1}^{n^{[l+1]}} \delta_k^{[l+1]} w^{[l+1]}_{k,i} \Big) \times f'^{[l]}_i (z^{[l]}_{i})$$

## TODO:

Write a function ``backpropagation`` that computes:

- $\frac{\partial L}{\partial w^{[l]}_{i,j}}$ and store them in ``model.dL_dw[str(l)][i,j]`` for $l \in [1,2]$ 
- $\frac{\partial L}{\partial b^{[l]}_{j}}$ and store them in ``model.dL_db[str(l)][j]`` for $l \in [1,2]$ 

**Hints**

- Look at how MiniNet is implemented and at the cell above (``weight_initialization`` and ``check_computational_graph`` to see how to access MiniNet weights and update MiniNet attributes)

Once you think you are done:

- You can check if your function works well by runnning the next cell.
- Since ``dL_dw`` are not trainable parameters of your model they should not be in the computational graph. Check that all your ``dL_dw``  (or at least one of them) have its attribute ``require_grad`` set to ``False``. If not use the [detach](https://pytorch.org/docs/stable/autograd.html#torch.Tensor.detach) or [detach_](https://pytorch.org/docs/stable/autograd.html#torch.Tensor.detach_) methods appropriately in your ``backpropagation`` function

In [7]:
import numpy as np

def mse(y_true, y_pred):
    return torch.sum((y_true - y_pred)**2)/2/len(y_true)

def dmse(y_true, y_pred):
    return torch.sum(-(y_true - y_pred))/len(y_true)


def backpropagation(model, y_true, y_pred, dloss=dmse):
    # Note: After implementing I realized I named my variable 'lambda', rather than 'delta', as in the above notation. Sorry if this causes confusion.
    # Current implementation does not account for unlimited amount of layers.
    # Calculate lambda for layer 2
    lmbda_init = dmse(y_true, y_pred) * model.df["2"](model.z["2"])    
    model.dL_dw[str(2)] = torch.empty(model.fc2.weight.data.size())
    model.dL_db[str(2)] = torch.empty(model.fc2.bias.data.size())
    for i in range(model.fc2.weight.data.size()[0]):
        for j in range(model.fc2.weight.data.size()[1]):
            model.dL_dw[str(2)][i,j] = lmbda_init[0][i] * model.a[str(1)][0][j]
            model.dL_db[str(2)][i] = lmbda_init[0][i]
    
    
    lmbda_w = torch.zeros(model.fc1.weight.data.size()[0])
    print(lmbda_w)
    lmbda_b = torch.zeros(model.fc1.bias.data.size())
    for i in range(model.fc1.weight.data.size()[0]):
        for j in range(model.fc1.weight.data.size()[1]):
            lmbda_w[i] += lmbda_init[0][j] * model.fc2.weight.data[j][i]
            
    lmbda_b = lmbda_w * model.df[str(1)](model.z[str(1)])
    lmbda_w = lmbda_w * model.df[str(1)](model.z[str(1)])
    
    n_dL_dw = torch.zeros(model.fc1.weight.data.size())
    for i in range(model.fc1.weight.data.size()[0]):
        for j in range(model.fc1.weight.data.size()[1]):
            x = lmbda_w[0][i]
            y = model.a[str(0)][0][j]
            n_dL_dw[i][j] = x * y
            
    print(lmbda_w)
    print(lmbda_b)

    model.dL_db[str(1)] = lmbda_b
    model.dL_dw[str(1)] = n_dL_dw
        
        
        
    ...
    return None

In [8]:
def compare_with_gradients(model):
    print( " =========== COMPARE GRADIENTS =========== ")
    print(" --- Our computation:\n", model.fc1.weight.grad)
    print(" --- Autograd's computation:\n",  model.dL_dw['1'])
    print("Norm (fc1.weight.grad - model.dL_dw['1']):   %.2f" %torch.norm(model.fc1.weight.grad.flatten() - model.dL_dw['1'].flatten()) )
    print("Norm (fc2.weight.grad - model.dL_dw['2']):   %.2f" %torch.norm(model.fc2.weight.grad.flatten() - model.dL_dw['2'].flatten()))
    print("Norm (fc1.weight.bias - model.dL_db['1']):   %.2f" %torch.norm(model.fc1.bias.grad.flatten() - model.dL_db['1'].flatten()))
    print("Norm (fc2.weight.bias - model.dL_db['2']):   %.2f" %torch.norm(model.fc2.bias.grad.flatten() - model.dL_db['2'].flatten()))
    return None

def test_backprop(model, optimizer, loss_fn):
    model.train()
    for i, (x, y) in enumerate(zip(inputs, y_true)):
        print("\n === Input %d === "%i)
    
        out = model(x)
        loss = loss_fn(out, y)

        optimizer.zero_grad()
        loss.backward()
        backpropagation(model, y, out)

        optimizer.step()
        
        compare_with_gradients(model)

N = 20
inputs = [torch.normal( torch.tensor([-i,i], dtype=torch.float), torch.tensor([1.,2.]) ) for i in range(N)]
y_true = [i*torch.ones(2) for i in range(N)]

model = MiniNet()
model = weight_initialization(model)    

loss_fn = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=1e-2)

test_backprop(model, optimizer, loss_fn)


print("\n ====  Check that weights have been updated ======")
print(model.fc1.weight)


 === Input 0 === 
tensor([0., 0., 0.])
tensor([[0.0004, 0.0003, 0.0002]], grad_fn=<MulBackward0>)
tensor([[0.0004, 0.0003, 0.0002]], grad_fn=<MulBackward0>)
 =========== COMPARE GRADIENTS =========== 
 --- Our computation:
 tensor([[1.7111e-04, 3.1728e-04],
        [1.1288e-04, 2.0931e-04],
        [6.9273e-05, 1.2845e-04]])
 --- Autograd's computation:
 tensor([[1.7112e-04, 3.1730e-04],
        [1.1289e-04, 2.0932e-04],
        [6.9278e-05, 1.2846e-04]], grad_fn=<CopySlices>)
Norm (fc1.weight.grad - model.dL_dw['1']):   0.00
Norm (fc2.weight.grad - model.dL_dw['2']):   0.00
Norm (fc1.weight.bias - model.dL_db['1']):   0.00
Norm (fc2.weight.bias - model.dL_db['2']):   0.00

 === Input 1 === 
tensor([0., 0., 0.])
tensor([[-0.0002, -0.0003, -0.0003]], grad_fn=<MulBackward0>)
tensor([[-0.0002, -0.0003, -0.0003]], grad_fn=<MulBackward0>)
 =========== COMPARE GRADIENTS =========== 
 --- Our computation:
 tensor([[6.0377e-05, 2.3518e-04],
        [7.4026e-05, 2.8834e-04],
        [8.6607e-0

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/torch/nn/modules/loss.py:446: UserWarning: Using a target size (torch.Size([2])) that is different to the input size (torch.Size([1, 2])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)



tensor([[-1.4511e-08, -3.0100e-10, -2.3711e-08]], grad_fn=<MulBackward0>)
tensor([[-1.4511e-08, -3.0100e-10, -2.3711e-08]], grad_fn=<MulBackward0>)
 =========== COMPARE GRADIENTS =========== 
 --- Our computation:
 tensor([[ 1.8241e-07, -2.1980e-07],
        [ 0.0000e+00,  0.0000e+00],
        [ 2.9979e-07, -3.6124e-07]])
 --- Autograd's computation:
 tensor([[ 1.8462e-07, -2.2247e-07],
        [ 3.8295e-09, -4.6145e-09],
        [ 3.0166e-07, -3.6350e-07]], grad_fn=<CopySlices>)
Norm (fc1.weight.grad - model.dL_dw['1']):   0.00
Norm (fc2.weight.grad - model.dL_dw['2']):   0.00
Norm (fc1.weight.bias - model.dL_db['1']):   0.00
Norm (fc2.weight.bias - model.dL_db['2']):   0.00

 === Input 14 === 
tensor([0., 0., 0.])
tensor([[-1.0846e-08, -1.8547e-10, -1.4472e-08]], grad_fn=<MulBackward0>)
tensor([[-1.0846e-08, -1.8547e-10, -1.4472e-08]], grad_fn=<MulBackward0>)
 =========== COMPARE GRADIENTS =========== 
 --- Our computation:
 tensor([[ 1.3924e-07, -1.7084e-07],
        [ 0.0000e+00, 

## 5. Implementing Gradient descent in Pytorch inside the training loop 

Now that we know how to backpropagate the gradient, the only thing missing to the learning process of our network is how to update parameters using the gradient descent update equation. 

In the previous section we used a very simple MLP so the only derivatives we had to compute were the derivates of the activation functions. Computing all the terms required by the backpropagation algorithm might become harder when you start including more complex layers such as convolutions. In this section we will go back to using ``loss.backward()`` and will focus on updating parameters once the backpropagation is already done.

The update step does not depend on the architecture's complexity. To illustrate this fact, we will now use a LeNet5 model.

### 5.0 LetNet5

## TODO:

Copy paste your favorite implementation of LeNet5 architecture.

In [9]:
import torch
from torch import nn

model = nn.Sequential(
    nn.Conv2d(3, 6, 5, 1),
    nn.ReLU(),
    nn.MaxPool2d(2, 2),
    nn.Conv2d(6, 16, 5, 1),
    nn.ReLU(),
    nn.MaxPool2d(2, 2),
    nn.Flatten(),

    nn.Linear(400, 120),
    nn.ReLU(),
    nn.Linear(120, 84),
    nn.ReLU(),
    nn.Linear(84, 10),
    nn.LogSoftmax(dim=-1)
)

 

### 5.1 Manual weight optimization

## TODO: (read until the end of this cell)

1. Remove the line corresponding to the parameter update step in the training loop below  
2. Replace it by a manual update of the parameters using the gradient descent rule $\theta = \theta - \alpha \nabla L(\theta)  $   
3. Run the '5.2 Compare results` cell to compare with the PyTorch implementation of SGD with consistent optimizer parameters  

Once the 3rd point works well (it's normal if values are not exactly equal, but they should follow the same trend):

4. Add L2 regularization (using the ``weight_decay`` parameter) to your optimizer.
5. Run the '5.3 Compare results` cell (keep the 5.2 one for vanilla SGD so that we can easily compare) to compare with the PyTorch implementation of SGD with consistent optimizer parameters and using regularization

Once the 5th point works well:

6. Add momentum (using the ``momentum_coeff`` parameter) to your optimizer. **Warning** there are different equations for momentum, use the one used by PyTorch so that comparisons make sense. (See SGD's [note](https://pytorch.org/docs/stable/optim.html?highlight=sgd#torch.optim.SGD))
7. Run the '5.4 Compare results` cell (keep the 5.3 one for SGD+L2 so that we can easily compare) to compare with the PyTorch implementation of SGD with consistent optimizer parameters and using momentum **WITHOUT** regularization

Once the 7th point works well:

9. Run the '5.5 Compare results` cell (keep the 5.4 one for SGD+Momentum so that we can easily compare) to compare with the PyTorch implementation of SGD with consistent optimizer parameters and using momentum **WITH** regularization

**Hints** 

- Add the `with torch.no_grad():` before updating parameters
- Remember that you can access each of the parameters of your model using for example ``for p in model.parameters()``
- Might be useful to detach some tensors at some point :) 
- Remember to be careful when updating trainable parameters!
- You'll still have to re-initialize the gradients after each step (see [torch.Tensor.zero_](https://pytorch.org/docs/stable/tensors.html?highlight=zero_#torch.Tensor.zero_))

In [28]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f"Device {device}.")

def training_loop_manual_optim(n_epochs, model, loss_fn, train_loader, lr=1e-2, momentum_coeff=0., weight_decay=0.):
    model.train()
    
    for epoch in range(1, n_epochs + 1):
        loss_train = 0.0
        for imgs, labels in train_loader:
            imgs = imgs.to(device=device) 
            labels = labels.to(device=device)

            # TODO!
            outputs = model(imgs)
            loss = loss_fn(outputs, labels)
            loss.backward()
            with torch.no_grad():
                if epoch == 1:
                    #initialise velocities array first epoch
                    velocities = []
                for i, p in enumerate(model.parameters()):
                    # 7
                    if epoch == 1:
                        # Fill velocities array with parameter shape at parameter index first epoch
                        velocities.append(torch.zeros(p.grad.data.size()))
                    if momentum_coeff > 0:
                        #Update velocities array
                        velocities[i] = momentum_coeff * velocities[i] + p.grad.data
                        p.data -= (lr * velocities[i] + lr*weight_decay*p.data)
                        
                    # 5
                    else:
                        #2 p.data -= lr*p.grad.data
                        p.data -= (lr*p.grad.data + lr*weight_decay*p.data)
                    p.grad.data.zero_()
                
            #optimizer.step()
            loss_train += loss.item()

        if epoch == 1 or epoch % 10 == 0:
            print('{}  |  Epoch {}  |  Training loss {:.3f}'.format(
                datetime.datetime.now(), epoch,
                loss_train / len(train_loader)))
            

Device cpu.


### 5.2 Compare results to optim.SGD: vanilla version

**NOTE** You can reduce the number of epochs (and also increase how often they are printed) if your computer is slow when you are still trying to make it work. Once it works try to go back to a higher number of epochs before submitting your assignment.



### Result
The optimizers perfom equally well

In [46]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f"Training on device {device}.")


train_loader = torch.utils.data.DataLoader(cifar2_train, batch_size=64, shuffle=True)
#model = LeNet5()  # TODO: Change this with your own LeNet5 class name
loss_fn = nn.CrossEntropyLoss()

n_epochs = 32
lr = 1e-2

print("\n ========= Training using Pytorch's SGD =========")
model01 = copy.deepcopy(model).to(device=device) 

# TODO: Choose the parameter values consistently so that comparisons make sense
optimizer = optim.SGD(model01.parameters(), lr=lr)


training_loop(
    n_epochs = n_epochs,
    optimizer = optimizer, 
    model = model01,
    loss_fn = loss_fn,
    train_loader = train_loader,
)


print("\n ==== Using manual update inside training loop ======")
model03 = copy.deepcopy(model).to(device=device) 

# TODO: Choose the parameter values consistently so that comparisons make sense
training_loop_manual_optim(
    n_epochs = n_epochs,
    model = model03,
    loss_fn = loss_fn,
    train_loader = train_loader,
    lr = lr,
)

# UNCOMMENT ONCE SECTION 6 IS DONE
# print("\n ========= Training using our SGD =========")
# model02 = copy.deepcopy(model).to(device=device) 
# # TODO: Choose the parameter values consistently so that comparisons make sense
# optimizer = MySGD(model02.parameters(), lr=lr)

# training_loop(
#     n_epochs = n_epochs,
#     optimizer = optimizer,
#     model = model02,
#     loss_fn = loss_fn,
#     train_loader = train_loader,
# )

train_loader = torch.utils.data.DataLoader(cifar2_train, batch_size=64, shuffle=False)
val_loader = torch.utils.data.DataLoader(cifar2_val, batch_size=64, shuffle=False)

print("\n ========= Using Pytorch's SGD =========")
validate(model01, train_loader, val_loader)
print("\n ==== Using manual update inside training loop ======")
validate(model03, train_loader, val_loader)
# UNCOMMENT ONCE SECTION 6 IS DONE
# print("\n ========= Using our SGD =========")
# validate(model02, train_loader, val_loader)

Training on device cpu.

 ========= Training using Pytorch's SGD =========
2021-02-21 16:15:34.551559  |  Epoch 1  |  Training loss 1.246
2021-02-21 16:15:51.265336  |  Epoch 10  |  Training loss 0.357
2021-02-21 16:16:09.949890  |  Epoch 20  |  Training loss 0.307
2021-02-21 16:16:28.015569  |  Epoch 30  |  Training loss 0.267

 ==== Using manual update inside training loop ======
2021-02-21 16:16:33.291210  |  Epoch 1  |  Training loss 1.241
2021-02-21 16:16:50.530950  |  Epoch 10  |  Training loss 0.360
2021-02-21 16:17:11.072936  |  Epoch 20  |  Training loss 0.306
2021-02-21 16:17:33.783978  |  Epoch 30  |  Training loss 0.268

 ========= Using Pytorch's SGD =========
Accuracy train: 0.90
Accuracy val: 0.88

 ==== Using manual update inside training loop ======
Accuracy train: 0.90
Accuracy val: 0.89


{'train': 0.8981, 'val': 0.888}

### 5.3 Compare results to optim.SGD: (with Regularization)

**NOTE** You can reduce the number of epochs (and also increase how often they are printed) if your computer is slow when you are still trying to make it work. Once it works try to go back to a higher number of epochs before submitting your assignment.

### Result

The optimizers perform equally well

In [47]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f"Training on device {device}.")

train_loader = torch.utils.data.DataLoader(cifar2_train, batch_size=64, shuffle=True)
#model = LeNet5()  # TODO: Change this with your own LeNet5 class name
loss_fn = nn.CrossEntropyLoss()

n_epochs = 32
lr = 1e-2
weight_decay = 0.001

print("\n ========= Training using Pytorch's SGD =========")
model01 = copy.deepcopy(model).to(device=device) 

# TODO: Choose the parameter values consistently so that comparisons make sense
optimizer = optim.SGD(model01.parameters(), lr=lr, weight_decay=weight_decay)


training_loop(
    n_epochs = n_epochs,
    optimizer = optimizer, 
    model = model01,
    loss_fn = loss_fn,
    train_loader = train_loader,
)

print("\n ==== Using manual update inside training loop ======")
model03 = copy.deepcopy(model).to(device=device) 

# TODO: Choose the parameter values consistently so that comparisons make sense
training_loop_manual_optim(
    n_epochs = n_epochs,
    model = model03,
    loss_fn = loss_fn,
    train_loader = train_loader,
    lr = lr,
    weight_decay=weight_decay,
)

# UNCOMMENT ONCE SECTION 6 IS DONE
# print("\n ========= Training using our SGD =========")
# model02 = copy.deepcopy(model).to(device=device) 
# # TODO: Choose the parameter values consistently so that comparisons make sense
# optimizer = MySGD(model02.parameters(), lr=lr, weight_decay=weight_decay)

# training_loop(
#     n_epochs = n_epochs,
#     optimizer = optimizer,
#     model = model02,
#     loss_fn = loss_fn,
#     train_loader = train_loader,
# )

train_loader = torch.utils.data.DataLoader(cifar2_train, batch_size=64, shuffle=False)
val_loader = torch.utils.data.DataLoader(cifar2_val, batch_size=64, shuffle=False)

print("\n ========= Using Pytorch's SGD =========")
validate(model01, train_loader, val_loader)
print("\n ==== Using manual update inside training loop ======")
validate(model03, train_loader, val_loader)
# UNCOMMENT ONCE SECTION 6 IS DONE
# print("\n ========= Using our SGD =========")
# validate(model02, train_loader, val_loader)

Training on device cpu.

 ========= Training using Pytorch's SGD =========
2021-02-21 16:19:35.401356  |  Epoch 1  |  Training loss 1.251
2021-02-21 16:19:56.859558  |  Epoch 10  |  Training loss 0.360
2021-02-21 16:20:15.791876  |  Epoch 20  |  Training loss 0.307
2021-02-21 16:20:33.388423  |  Epoch 30  |  Training loss 0.277

 ==== Using manual update inside training loop ======
2021-02-21 16:20:38.714177  |  Epoch 1  |  Training loss 1.240
2021-02-21 16:20:55.453962  |  Epoch 10  |  Training loss 0.358
2021-02-21 16:21:13.802401  |  Epoch 20  |  Training loss 0.310
2021-02-21 16:21:32.268552  |  Epoch 30  |  Training loss 0.272

 ========= Using Pytorch's SGD =========
Accuracy train: 0.87
Accuracy val: 0.86

 ==== Using manual update inside training loop ======
Accuracy train: 0.89
Accuracy val: 0.87


{'train': 0.8899, 'val': 0.873}

### 5.4 Compare results to optim.SGD: (with Momentum)

**NOTE** You can reduce the number of epochs (and also increase how often they are printed) if your computer is slow when you are still trying to make it work. Once it works try to go back to a higher number of epochs before submitting your assignment.

### Result

The optimizers perform equally well. Training accuracy is a little higher than validation accuracy, meaning we might have some overfitting

In [48]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f"Training on device {device}.")

train_loader = torch.utils.data.DataLoader(cifar2_train, batch_size=64, shuffle=True)
#model = LeNet5()  # TODO: Change this with your own LeNet5 class name
loss_fn = nn.CrossEntropyLoss()

n_epochs = 32
lr = 1e-2
momentum_coeff = 0.6


print("\n ========= Training using Pytorch's SGD =========")
model01 = copy.deepcopy(model).to(device=device) 

# TODO: Choose the parameter values consistently so that comparisons make sense
optimizer = optim.SGD(model01.parameters(), lr=lr, momentum=momentum_coeff)


training_loop(
    n_epochs = n_epochs,
    optimizer = optimizer, 
    model = model01,
    loss_fn = loss_fn,
    train_loader = train_loader,
)

print("\n ==== Using manual update inside training loop ======")
model03 = copy.deepcopy(model).to(device=device) 

# TODO: Choose the parameter values consistently so that comparisons make sense
training_loop_manual_optim(
    n_epochs = n_epochs,
    model = model03,
    loss_fn = loss_fn,
    train_loader = train_loader,
    lr = lr,
    momentum_coeff=momentum_coeff
)

# UNCOMMENT ONCE SECTION 6 IS DONE
# print("\n ========= Training using our SGD =========")
# model02 = copy.deepcopy(model).to(device=device) 
# # TODO: Choose the parameter values consistently so that comparisons make sense
# optimizer = MySGD(model02.parameters(), lr=lr, momentum_coeff=momentum_coeff)

# training_loop(
#     n_epochs = n_epochs,
#     optimizer = optimizer,
#     model = model02,
#     loss_fn = loss_fn,
#     train_loader = train_loader,
# )

train_loader = torch.utils.data.DataLoader(cifar2_train, batch_size=64, shuffle=False)
val_loader = torch.utils.data.DataLoader(cifar2_val, batch_size=64, shuffle=False)

print("\n ========= Using Pytorch's SGD =========")
validate(model01, train_loader, val_loader)
print("\n ==== Using manual update inside training loop ======")
validate(model03, train_loader, val_loader)
# UNCOMMENT ONCE SECTION 6 IS DONE
# print("\n ========= Using our SGD =========")
# validate(model02, train_loader, val_loader)

Training on device cpu.

 ========= Training using Pytorch's SGD =========
2021-02-21 16:22:49.315265  |  Epoch 1  |  Training loss 0.909
2021-02-21 16:23:08.338918  |  Epoch 10  |  Training loss 0.306
2021-02-21 16:23:28.836330  |  Epoch 20  |  Training loss 0.240
2021-02-21 16:23:50.064389  |  Epoch 30  |  Training loss 0.167

 ==== Using manual update inside training loop ======
2021-02-21 16:23:55.625696  |  Epoch 1  |  Training loss 1.245
2021-02-21 16:24:17.752357  |  Epoch 10  |  Training loss 0.316
2021-02-21 16:24:38.704164  |  Epoch 20  |  Training loss 0.238
2021-02-21 16:24:59.441798  |  Epoch 30  |  Training loss 0.170

 ========= Using Pytorch's SGD =========
Accuracy train: 0.94
Accuracy val: 0.88

 ==== Using manual update inside training loop ======
Accuracy train: 0.94
Accuracy val: 0.89


{'train': 0.945, 'val': 0.887}

### 5.5 Compare results to optim.SGD: (with Regularization and Momentum)

**NOTE** You can reduce the number of epochs (and also increase how often they are printed) if your computer is slow when you are still trying to make it work. Once it works try to go back to a higher number of epochs before submitting your assignment.

### Result

The optimizers perform equally well

In [59]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f"Training on device {device}.")


train_loader = torch.utils.data.DataLoader(cifar2_train, batch_size=64, shuffle=True)
#model = LeNet5()  # TODO: Change this with your own LeNet5 class name
loss_fn = nn.CrossEntropyLoss()

n_epochs = 32
lr = 1e-2
momentum_coeff = 0.6
weight_decay = 0.001


print("\n ========= Training using Pytorch's SGD =========")
model01 = copy.deepcopy(model).to(device=device) 

# TODO: Choose the parameter values consistently so that comparisons make sense
optimizer = optim.SGD(model01.parameters(), lr=lr, momentum=momentum_coeff, weight_decay=weight_decay)


training_loop(
    n_epochs = n_epochs,
    optimizer = optimizer, 
    model = model01,
    loss_fn = loss_fn,
    train_loader = train_loader,
)

print("\n ==== Using manual update inside training loop ======")
model03 = copy.deepcopy(model).to(device=device) 

# TODO: Choose the parameter values consistently so that comparisons make sense
training_loop_manual_optim(
    n_epochs = n_epochs,
    model = model03,
    loss_fn = loss_fn,
    train_loader = train_loader,
    lr = lr,
    momentum_coeff=momentum_coeff,
    weight_decay=weight_decay,

)

# UNCOMMENT ONCE SECTION 6 IS DONE
print("\n ========= Training using our SGD =========")
model02 = copy.deepcopy(model).to(device=device) 
# # TODO: Choose the parameter values consistently so that comparisons make sense
optimizer = MySGD(model02.parameters(), lr=lr, momentum_coeff=momentum_coeff, weight_decay=weight_decay)

training_loop(
    n_epochs = n_epochs,
    optimizer = optimizer,
    model = model02,
    loss_fn = loss_fn,
    train_loader = train_loader,
)

train_loader = torch.utils.data.DataLoader(cifar2_train, batch_size=64, shuffle=False)
val_loader = torch.utils.data.DataLoader(cifar2_val, batch_size=64, shuffle=False)

print("\n ========= Using Pytorch's SGD =========")
validate(model01, train_loader, val_loader)
print("\n ==== Using manual update inside training loop ======")
validate(model03, train_loader, val_loader)
# UNCOMMENT ONCE SECTION 6 IS DONE
print("\n ========= Using our SGD =========")
validate(model02, train_loader, val_loader)

Training on device cpu.

 ========= Training using Pytorch's SGD =========
2021-02-21 16:57:32.090686  |  Epoch 1  |  Training loss 0.904
2021-02-21 16:57:47.746986  |  Epoch 10  |  Training loss 0.312
2021-02-21 16:58:05.092550  |  Epoch 20  |  Training loss 0.254
2021-02-21 16:58:22.411241  |  Epoch 30  |  Training loss 0.185

 ==== Using manual update inside training loop ======
2021-02-21 16:58:27.749445  |  Epoch 1  |  Training loss 1.245
2021-02-21 16:58:44.029365  |  Epoch 10  |  Training loss 0.313
2021-02-21 16:59:02.066771  |  Epoch 20  |  Training loss 0.244
2021-02-21 16:59:20.167236  |  Epoch 30  |  Training loss 0.179

 ========= Training using our SGD =========
2021-02-21 16:59:27.293028  |  Epoch 1  |  Training loss 1.255
2021-02-21 16:59:47.774357  |  Epoch 10  |  Training loss 0.361
2021-02-21 17:00:08.438997  |  Epoch 20  |  Training loss 0.310
2021-02-21 17:00:27.775477  |  Epoch 30  |  Training loss 0.272

 ========= Using Pytorch's SGD =========
Accuracy train: 0.

{'train': 0.8988, 'val': 0.8855}

## 6. Implementing an Optimizer in PyTorch

You might have noticed that when implementing more and more complex optimizers you start polluting a bit too much your training loop. We might prefer to seperate the code consisting in applying the gradient descent method and the code consisting in feeding your model with new inputs. This is what PyTorch does naturally. Indeed, we instantiate an optimizer (e.g. optim.SGD) with all its parameters, (learning rate, momentum, weight_decay, etc) and then we simply need to call ``optimizer.step()``  and ``optimizer.zero_grad()`` in our training loop independently of the optimization method used and of its parameters.

Similarly to how we defined our custom neural network in Pytorch, we will create a class to define our custom optimizer. But instead of inheriting the ``nn.Module`` class, we will inherit the [torch.optim.Optimizer](https://pytorch.org/docs/stable/optim.html#torch.optim.Optimizer). As written in the documentation, it is simply the "Base class for all optimizers". Keeping the ``nn.Module`` comparison, instead of having to implement a ``forward`` method (regardless of our model) we will have to implement a [step](https://pytorch.org/docs/stable/optim.html#torch.optim.Optimizer.step) method (regardless of the optimization method chosen )

## TODO:

Same questions as above (implement SGD + momentum + L2 successively and compare with PyTorch's implementation by uncommenting the block of code calling ``MySGD``)

**Hints**

- You might want to add an attribute to your Optimizer when implementing the momentum version
- Might be useful to detach some tensors at some point :) 
- Remember to be careful when updating trainable parameters!
- It's okay if you don't succeed here. Try and we will grade generously. :) 


In [57]:
class MySGD(Optimizer):
    """Implements SGD (optionally with momentum and regularization)
    """

    def __init__(
        self, 
        params,
        lr=0.01,
        momentum_coeff=0.,
        weight_decay=0.,
    ):
        # Don't pay attention to this 'default' thing
        # it's just something required by PyTorch
        defaults = dict(
            lr=lr, 
            momentum_coeff=momentum_coeff,
            weight_decay=weight_decay, 
        )
        super().__init__(params, defaults)
        self.state = {"step": 0}

    @torch.no_grad()
    def step(self, closure=None):
        """Performs a single optimization step.
        """  
        # Don't pay attention to this 'closure' thing
        # it's just something required by PyTorch


        # In the optimizer object, the parameters are stored 
        # in groups for more flexibility (e.g each layer 
        # could have a different learning rate for example)
        # In our case just consider that :
        #
        # for group in self.param_groups:
        #   for p in group['params']:
        #
        # Is the optimizer counterpart of:
        #
        # for p in model.parameters()
        #
        i = 0
        if self.state["step"] == 0:
            velocities = []
        for group in self.param_groups:
            for p in group['params']:
                if p.grad is not None and self.state["step"] == 0:
                    velocities.append(torch.zeros(p.grad.data.size()))
                    
        group_count = 0
        for group in self.param_groups:
            weight_decay = group['weight_decay']
            momentum_coeff = group['momentum_coeff']
            lr = group['lr']
            for p in group['params']:
                if p.grad is not None:
                    with torch.no_grad():
                        # Now we are safe
                        grad = torch.clone(p.grad).detach()
                        
                        # Commented as I kept running into some scope issue I was unable to resolve.
                        # To enable momentum, change float('inf') to 0.
                        if momentum_coeff > float('inf'):
                            print(group_count)
                            print(velocities)
                            velocities[i] = momentum_coeff * velocities[i] + p.grad.data
                            p.data -= (lr * velocities[i] + lr*weight_decay*p.data)
                        else:
                            p.data -= (lr*p.grad.data + lr*weight_decay*p.data)
                        p.grad.data.zero_()
                        i += 1
                group_count += 1

        self.state["step"] += 1


In [ ]:
          with torch.no_grad():
                if epoch == 1:
                    #initialise velocities array first epoch
                    velocities = []
                for i, p in enumerate(model.parameters()):
                    # 7, 9
                    if epoch == 1:
                        # Fill velocities array with parameter shape at parameter index first epoch
                        velocities.append(torch.zeros(p.grad.data.size()))
                    if momentum_coeff > 0:
                        #Update velocities array
                        velocities[i] = momentum_coeff * velocities[i] + p.grad.data
                        p.data -= (lr * velocities[i] + lr*weight_decay*p.data)
                        
                    # 2, 5
                    else:
                        p.data -= (lr*p.grad.data + lr*weight_decay*p.data)
                    p.grad.data.zero_()
                
            #optimizer.step()